<a href="https://colab.research.google.com/github/ranjithrece/ranjithrece/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Chatbot**

we are going to built a Chatbot with [keras](https://keras.io/)

You can get the predefined pattern and response  file here [download](https://drive.google.com/file/d/1v_kEDymTKR4YBsl1l6RLw4OHpXMuIQXz/view?usp=sharing) 

In [ ]:
#importing neccesary modules
import nltk

from nltk.stem import WordNetLemmatizer
import numpy as np
import json
import pickle
import random


from keras.layers import Dense,Dropout
from keras.models import Sequential

In [93]:
lem = WordNetLemmatizer()#create a inistane for lemmitizer
intents = json.loads(open('intents.json').read())#loading the intents from json file

#creating a empty list for seprating values
words = []
classes = set()
docs = []
ignore_wrds = ['?','!']


In [94]:
#content - which type of question
#pattern - user question
#response - prestored response for user questions

for intent in intents["intents"]:#looping trough the intent 
  for pattern in intent['patterns']:

    w = nltk.word_tokenize(pattern)# tokenizing the words
    words.extend(w)

    docs.append([w,intent['tag']])#creating a doc with tokenized words and their tag

    classes.add(intent['tag'])

In [95]:
words = [lem.lemmatize(w.lower()) for w in words if w not in ignore_wrds]#lemmatizing the words eg. running to the base form of -> run

classes = sorted(list(classes))#extracting the unique values
words = sorted(list(set(words)))

In [96]:
pickle.dump(words,open('words.pkl','wb'))#saving values in byte format
pickle.dump(classes,open('classes.pkl','wb'))

In [97]:
#creating a bag of words(one-hot) for training
training  = []


for doc in docs:

    bag =[]
    pattern_wrds = doc[0] #extracting the text
    pattern_wrds = [lem.lemmatize(word.lower()) for word in pattern_wrds] #lemmatizing it

    for w in words:  #loop through the words and creating bag
      bag.append(1) if w in pattern_wrds else bag.append(0)

    out = np.zeros(len(words))
  
    out[classes.index(doc[1])] = 1 #assg 1 to the respective tag in classes

    training.append([bag,out])#appending the bag of words and bag of tag 
random.shuffle(training)
training = np.array(training)
train_x = training[:,0]#bag of words
train_y = training[:,1]#bag of tag

##Model

In [105]:
model = Sequential(name='Chatbot_model')
model.add(Dense(256,input_shape = (train_x.shape[1],),activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(train_y.shape[1],activation = 'softmax'))

model.summary()

Model: "Chatbot_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 256)               22784     
_________________________________________________________________
dropout_23 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_24 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_25 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 88)              

In [106]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])

hist = model.fit(train_x,train_y,epochs=200,batch_size=5)

Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 4.4052 - accuracy: 0.0336 
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 4.2498 - accuracy: 0.1166
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 3.8597 - accuracy: 0.3660
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 3.3791 - accuracy: 0.1190
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 2.6294 - accuracy: 0.1986
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1400 - accuracy: 0.2412
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.7255 - accuracy: 0.5439
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6240 - accuracy: 0.4913
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4196 - accuracy: 0.5876
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.1540 - accuracy: 0.6666
Epoch 11

In [107]:
model.save('model.h5',hist)#saving the model
print('model saved')

model saved


##Predicting 

In [108]:
def clean_up(sent):
  return  [lem.lemmatize(w.lower()) for w in  nltk.word_tokenize(sent)]

In [109]:
def bow(sentence):#creating a bow for input sent
  sentence = sentence
  bow = np.zeros(len(words))

  for s in sentence:
    for i,w in enumerate(words):
      if w == s:
        bow [i] = 1 

  return(bow)

In [110]:
def predict(sent,model):
  p = bow(sent)
  pred = model.predict(np.array([p]))[0]
  
  thrsh = 0.30 #leave the predictions below this thrshold
  res = [[i,j] for i,j in enumerate(pred) if j > thrsh]
  res.sort(key= lambda x:x[1],reverse=True)#sorting the predictions
  result_lis = []
  for key,val in res:
    result_lis.append({'intent':classes[key],'prob':str(val)})

  return result_lis

In [111]:
predict('Hi there Hello',model)

[{'intent': 'greeting', 'prob': '0.53335714'},
 {'intent': 'goodbye', 'prob': '0.3478233'}]

In [112]:
def get_response(intn,int_json):
  
  tag = intn[0]['intent']
  listOfTag = int_json['intents']
  for i in listOfTag:
    if (tag == i['tag']):
      pred = random.choice(i['responses'])
      break
  return pred

In [113]:
def chatbot(msg):
  int = predict(msg,model)
  res = get_response(int,intents)
  return res

In [114]:
chatbot("Hi")#getting response

'Good to see you again'

## Chatbot gui
###Note:
      Colab can't run below code code.

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()